In [1]:
import numba 
from numba import float64
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [2]:
import ctrl
import electric

vsc = electric.vsc()
vsrc = electric.source3ph()
    

In [11]:
import ctrl
import electric
 
    
@numba.jit(nopython=True)
def run():
    dt = 50e-6
    t_end = 0.2
    decimation = 1
    N_steps = int(t_end/dt)
    N_saves = int(t_end/dt/decimation)
    T = np.zeros((N_saves,1))
    X = np.zeros((N_saves,3))
    Y = np.zeros((N_saves,7))
    Y1 = np.zeros((N_saves,3))
    
    vsc = electric.vsc()
    vsrc = electric.source3ph()
    
    t = 0.0
    x_0 = vsc.x_0
    x = x_0
    y = np.zeros(())

    u_vsc= np.zeros((7,1))
    X[0,:] = x[:,0]
    Y[0,:] = vsc.h_eval(t,u_vsc,x,y)[:,0]

    it = 0
    it_decimation = 0
    it_save = 0
    solver = 1
    
    f = np.zeros((2,1))    
    vsc.C = 2200e-2
    u_vsc[6] = 0.0
    
    for it in range(N_steps):   
        if t > 0.1:
            u_vsc[6] = 0.0
            
        t += dt        

        if solver == 1:
            # solver trapezoidal 1 step

            u_vsc[0:3] = vsrc.h_eval(t)/400.0*0.9
            u_vsc[3:6] = vsrc.h_eval(t)
            #u_vsc[3] = u_vsc[3]*0.95
            f = vsc.f_eval(t,u_vsc,x,y) 

            x = x + dt*(f)

        if it_decimation >= decimation:
            it_save += 1  
            X[it_save,:] = x[:,0]
            Y[it_save,:] = vsc.h_eval(t,u_vsc,x,y)[:,0]
            Y1[it_save,:] = u_vsc[0:3,0]
            T[it_save,:] = t
            it_decimation = 0
            
        it_decimation += 1              
        
        
    return T,X,Y,Y1
   
%timeit run()

T,X,Y,Y1 = run()

The slowest run took 67.94 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 13.3 ms per loop


In [12]:
fig1 = figure(width=800, height=300)
fig1.line(x=T[:,0], y=Y[:,0], color="red")
fig1.line(x=T[:,0], y=Y[:,1], color="green")
fig1.line(x=T[:,0], y=Y[:,2], color="blue")

fig2 = figure(width=800, height=300)
fig2.line(x=T[:,0], y=Y[:,3], color="red")

fig3 = figure(width=800, height=300)
fig3.line(x=T[:,0], y=Y[:,4], color="red")
fig3.line(x=T[:,0], y=Y[:,5], color="green")
fig3.line(x=T[:,0], y=Y[:,6], color="blue")

figs = gridplot([[fig1],[fig2],[fig3]])

show(figs)

In [15]:
Dt = 50e-6
t_0 = 0.1
t_1 = 0.14

idx_0 = int(t_0/Dt)
idx_1 = int(t_1/Dt)
N = idx_1 - idx_0

freq = np.fft.fftfreq(N, d=Dt)
fft = np.fft.rfft(Y[idx_0:idx_1,0])

fig = figure(width=800, height=300)
fig.vbar(x = freq[0:int(N/2)], y=np.abs(fft)/N*2, color="red")
show(fig)


AttributeError: unexpected attribute 'y' to VBar, possible attributes are bottom, fill_alpha, fill_color, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, tags, top, visible, width or x

In [6]:
fft.shape

(401,)

In [7]:
t = np.linspace(0.0,0.02,1000)
fig = figure(width=800, height=300)
fig.line(x=t, y=np.arcsin(np.sin(6*100*t))/(np.pi/2.0), color="red")
show(fig)

In [8]:

Dt = 50.0e-6
t = 4.0*Dt
t_1 = t
t_2 = t + Dt

c_1 =   np.arcsin(np.sin(2.0*np.pi*2500*t_1))/(np.pi/2.0)
c_2 =   np.arcsin(np.sin(2.0*np.pi*2500*t_2))/(np.pi/2.0)

u = np.zeros((7,1))
u[0] = -0.49
u[1] = 0
u[2] = 0
        
t_s = (t_2-t_1)/(c_2 - c_1)*(u[0,0] - c_1) + t_1

if u[0,0]>c_1 and u[0,0]<c_2:
    eta = -((t_s - t_1)*(-1.0) + (t_2-t_s)*( 1.0))/Dt

if u[0,0]<c_1 and u[0,0]>c_2:
    eta = -((t_s - t_1)*( 1.0) + (t_2-t_s)*(-1.0))/Dt

if u[0,0]>c_1 and u[0,0]>c_2:
    eta = 1.0
    
if u[0,0]<c_1 and u[0,0]<c_2:
    eta = -1.0
    
print('t_s = {:2.3g} s'.format(t_s))
print('eta = {:2.3g}'.format(eta))

fig = figure(width=800, height=300)
fig.line(x=[t_1,t_2], y=[c_1,c_2], color="red")
fig.line(x=[t_1,t_2], y=[u[0],u[0]], color="blue")
show(fig)



t_s = 0.000249 s
eta = -0.96


In [9]:
t_s


0.00024899999999999998

In [10]:
eta

-0.95999999999999885